In [2]:
import numpy as np
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

# from data_loading import sine_data_generation
# from utils import random_generator
# from data_loading import MinMaxScaler

from torch.utils.data import DataLoader


# from utils import extract_time


In [3]:
def train_test_divide (data_x, data_x_hat, data_t, data_t_hat, train_rate = 0.8):
  """ Divide train and test data for both original and synthetic data.
  
  Args:
    - data_x: original data
    - data_x_hat: generated data
    - data_t: original time
    - data_t_hat: generated time
    - train_rate: ratio of training data from the original data
  """
  # Divide train/test index (original data)
  no = len(data_x)
  idx = np.random.permutation(no)
  train_idx = idx[:int(no*train_rate)]
  test_idx = idx[int(no*train_rate):]
    
  train_x = [data_x[i] for i in train_idx]
  test_x = [data_x[i] for i in test_idx]
  train_t = [data_t[i] for i in train_idx]
  test_t = [data_t[i] for i in test_idx]      
    
  # Divide train/test index (synthetic data)
  no = len(data_x_hat)
  idx = np.random.permutation(no)
  train_idx = idx[:int(no*train_rate)]
  test_idx = idx[int(no*train_rate):]
  
  train_x_hat = [data_x_hat[i] for i in train_idx]
  test_x_hat = [data_x_hat[i] for i in test_idx]
  train_t_hat = [data_t_hat[i] for i in train_idx]
  test_t_hat = [data_t_hat[i] for i in test_idx]
  
  return train_x, train_x_hat, test_x, test_x_hat, train_t, train_t_hat, test_t, test_t_hat


def extract_time (data):
  """Returns Maximum sequence length and each sequence length.
  
  Args:
    - data: original data
    
  Returns:
    - time: extracted time information
    - max_seq_len: maximum sequence length
  """
  time = list()
  max_seq_len = 0
  for i in range(len(data)):
    max_seq_len = max(max_seq_len, len(data[i][:,0]))
    time.append(len(data[i][:,0]))
    
  return time, max_seq_len

def random_generator (batch_size, z_dim, T_mb, max_seq_len):
  """Random vector generation.
  
  Args:
    - batch_size: size of the random vector
    - z_dim: dimension of random vector
    - T_mb: time information for the random vector
    - max_seq_len: maximum sequence length
    
  Returns:
    - Z_mb: generated random vector
  """
  Z_mb = list()
  for i in range(batch_size):
    temp = np.zeros([max_seq_len, z_dim])
    temp_Z = np.random.uniform(0., 1, [T_mb[i], z_dim])
    temp[:T_mb[i],:] = temp_Z
    Z_mb.append(temp_Z)
  return Z_mb


def batch_generator(data, time, batch_size):
  """Mini-batch generator.
  
  Args:
    - data: time-series data
    - time: time information
    - batch_size: the number of samples in each batch
    
  Returns:
    - X_mb: time-series data in each batch
    - T_mb: time information in each batch
  """
  no = len(data)
  idx = np.random.permutation(no)
  train_idx = idx[:batch_size]     
            
  X_mb = list(data[i] for i in train_idx)
  T_mb = list(time[i] for i in train_idx)
  
  return X_mb, T_mb

In [4]:
def MinMaxScaler(data):
  """Min Max normalizer.
  
  Args:
    - data: original data
  
  Returns:
    - norm_data: normalized data
  """
  numerator = data - np.min(data, 0)
  denominator = np.max(data, 0) - np.min(data, 0)
  norm_data = numerator / (denominator + 1e-7)
  return norm_data


def sine_data_generation (no, seq_len, dim):
  """Sine data generation.
  
  Args:
    - no: the number of samples
    - seq_len: sequence length of the time-series
    - dim: feature dimensions
    
  Returns:
    - data: generated data
  """  
  # Initialize the output
  data = list()

  # Generate sine data
  for i in range(no):      
    # Initialize each time-series
    temp = list()
    # For each feature
    for k in range(dim):
      # Randomly drawn frequency and phase
      freq = np.random.uniform(0, 0.1)            
      phase = np.random.uniform(0, 0.1)
          
      # Generate sine signal based on the drawn frequency and phase
      temp_data = [np.sin(freq * j + phase) for j in range(seq_len)] 
      temp.append(temp_data)
        
    # Align row/column
    temp = np.transpose(np.asarray(temp))        
    # Normalize to [0,1]
    temp = (temp + 1)*0.5
    # Stack the generated data
    data.append(temp)
                
  return data
    

def real_data_loading (data_name, seq_len):
  """Load and preprocess real-world datasets.
  
  Args:
    - data_name: stock or energy
    - seq_len: sequence length
    
  Returns:
    - data: preprocessed data.
  """  
  assert data_name in ['stock','energy']
  
  if data_name == 'stock':
    ori_data = np.loadtxt('data/stock_data.csv', delimiter = ",",skiprows = 1)
  elif data_name == 'energy':
    ori_data = np.loadtxt('data/energy_data.csv', delimiter = ",",skiprows = 1)
        
  # Flip the data to make chronological data
  ori_data = ori_data[::-1]
  # Normalize the data
  ori_data = MinMaxScaler(ori_data)
    
  # Preprocess the dataset
  temp_data = []    
  # Cut data by sequence length
  for i in range(0, len(ori_data) - seq_len):
    _x = ori_data[i:i + seq_len]
    temp_data.append(_x)
        
  # Mix the datasets (to make it similar to i.i.d)
  idx = np.random.permutation(len(temp_data))    
  data = []
  for i in range(len(temp_data)):
    data.append(temp_data[idx[i]])
    
  return data

Define Class for Module Construction

In [20]:
class Time_GAN_module(nn.Module):
    """
    Class from which a module of the Time GAN Architecture can be constructed, 
    consisting of a n_layer stacked RNN layers and a fully connected layer
    
    input_size = dim of data (depending if module operates on latent or non-latent space)
    """
    def __init__(self, input_size, output_size, hidden_dim, n_layers, activation=torch.sigmoid):
        super(Time_GAN_module, self).__init__()

        # Parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.sigma = activation

        #Defining the layers
        # RNN Layer
        self.rnn = nn.GRU(input_size, hidden_dim, n_layers, batch_first=True)   
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)
        
    def forward(self, x):
    
            batch_size = x.size(0)

            # Initializing hidden state for first input using method defined below
            hidden = self.init_hidden(batch_size)

            # Passing in the input and hidden state into the model and obtaining outputs
            out, hidden = self.rnn(x, hidden)
        
            # Reshaping the outputs such that it can be fit into the fully connected layer
            out = out.contiguous().view(-1, self.hidden_dim)
            out = self.fc(out)
            
            if self.sigma == nn.Identity:
                idendity = nn.Identity()
                return idendity(out)
                
            out = self.sigma(out)
            
            # HIDDEN STATES WERDEN IN DER PAPER IMPLEMENTIERUNG AUCH COMPUTED, ALLERDINGS NICHT BENUTZT?
            
            return out, hidden
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        # We'll send the tensor holding the hidden state to the device we specified earlier as well
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim)
        return hidden

Parameters

In [47]:
input_size = 5 # sequence length = number of features
output_size = 20
hidden_dim = 40
n_layers = 1
gamma = 1

no, seq_len, dim = 12800, 24, 1

batch_size = 128
epoch = 100

Data Generation

In [14]:
data = sine_data_generation(no, seq_len, dim)
data = MinMaxScaler(data)
data = torch.Tensor(data)
data.shape

torch.Size([12800, 24, 1])

Create Modules

In [ ]:
# embedder: num_layers = num_layers, fully_connected dim = hidden_dim
# recovery: num_layers = num_layers, fully_connected dim = dim 
# generator: num layers = num_layers, fully_connected dim = hidden_dim
# supervisor: num_layers = num_layers-1, fully_connected dim = hidden_dim
# discriminator: num_layers = num_layers, fully_connected dim = 1

In [7]:
Embedder = Time_GAN_module(input_size=dim, output_size=hidden_dim, hidden_dim=hidden_dim, n_layers=n_layers)
Embedder

Time_GAN_module(
  (rnn): GRU(1, 20, num_layers=3, batch_first=True)
  (fc): Linear(in_features=20, out_features=20, bias=True)
)

In [8]:
Recovery = Time_GAN_module(input_size=hidden_dim, output_size=dim, hidden_dim=hidden_dim, n_layers=n_layers)
Recovery

Time_GAN_module(
  (rnn): GRU(20, 20, num_layers=3, batch_first=True)
  (fc): Linear(in_features=20, out_features=1, bias=True)
)

In [9]:
Generator = Time_GAN_module(input_size=dim, output_size=hidden_dim, hidden_dim=hidden_dim, n_layers=n_layers)
Generator

Time_GAN_module(
  (rnn): GRU(1, 20, num_layers=3, batch_first=True)
  (fc): Linear(in_features=20, out_features=20, bias=True)
)

In [10]:
Supervisor = Time_GAN_module(input_size=hidden_dim, output_size=hidden_dim, hidden_dim=hidden_dim, n_layers=n_layers)
Supervisor

Time_GAN_module(
  (rnn): GRU(20, 20, num_layers=3, batch_first=True)
  (fc): Linear(in_features=20, out_features=20, bias=True)
)

In [11]:
Discriminator = Time_GAN_module(input_size=hidden_dim, output_size=1, hidden_dim=hidden_dim, n_layers=n_layers, 
                               activation=nn.Identity)
Discriminator

Time_GAN_module(
  (rnn): GRU(20, 20, num_layers=3, batch_first=True)
  (fc): Linear(in_features=20, out_features=1, bias=True)
)

Create Optimizers

In [ ]:
embedder_optimizer = optim.Adam(Embedder.parameters(), lr=0.001)
recovery_optimizer = optim.Adam(Recovery.parameters(), lr=0.001)
supervisor_optimizer = optim.Adam(Recovery.parameters(), lr=0.001)
discriminator_optimizer = optim.Adam(Discriminator.parameters(), lr=0.001)
generator_optimizer = optim.Adam(Generator.parameters(), lr=0.001)

Data Loader

In [ ]:
loader = DataLoader(data, batch_size, shuffle=True)
X = next(iter(loader))
H, _ = Embedder(X.float())
H_re = torch.reshape(H, (batch_size, seq_len, hidden_dim))
H.shape, H_re.shape

In [ ]:
batch_size, seq_len, hidden_dim

In [17]:
random_data = random_generator(batch_size=batch_size, z_dim=dim, 
                                       T_mb=extract_time(data)[0], max_seq_len=extract_time(data)[1])

In [18]:
loader = DataLoader(data, batch_size, shuffle=True)

random_loader = DataLoader(random_data, batch_size, shuffle=True)

binary_cross_entropy_loss = nn.BCEWithLogitsLoss()

MSE_loss = nn.MSELoss()



In [ ]:
X = next(iter(loader))
X.shape

In [48]:
def TimeGAN(data, parameters):
  hidden_dim = parameters["hidden_dim"]
  num_layers = parameters["num_layers"]
  iterations = parameters["iterations"]
  batch_size = parameters["batch_size"]
  module = parameters["module"]
  epoch = parameters["epoch"]
  no, seq_len, dim = np.asarray(data).shape
  z_dim = dim
  gamma = 1

  Embedder = Time_GAN_module(input_size=z_dim, output_size=hidden_dim, hidden_dim=hidden_dim, n_layers=num_layers)
  Recovery = Time_GAN_module(input_size=hidden_dim, output_size=dim, hidden_dim=hidden_dim, n_layers=n_layers)
  Generator = Time_GAN_module(input_size=dim, output_size=hidden_dim, hidden_dim=hidden_dim, n_layers=n_layers)
  Supervisor = Time_GAN_module(input_size=hidden_dim, output_size=hidden_dim, hidden_dim=hidden_dim, n_layers=n_layers)
  Discriminator = Time_GAN_module(input_size=hidden_dim, output_size=1, hidden_dim=hidden_dim, n_layers=n_layers, activation=nn.Identity)

  embedder_optimizer = optim.Adam(Embedder.parameters(), lr=0.001)
  recovery_optimizer = optim.Adam(Recovery.parameters(), lr=0.001)
  supervisor_optimizer = optim.Adam(Recovery.parameters(), lr=0.001)
  discriminator_optimizer = optim.Adam(Discriminator.parameters(), lr=0.001)
  generator_optimizer = optim.Adam(Generator.parameters(), lr=0.001)

  # Embedding Network Training
  print('Start Embedding Network Training')
  for e in range(epoch): 
    for batch_index, X in enumerate(loader):
        
        MSE_loss = nn.MSELoss()
        
        H, _ = Embedder(X.float())
        H = torch.reshape(H, (batch_size, seq_len, hidden_dim))

        X_tilde, _ = Recovery(H)
        X_tilde = torch.reshape(X_tilde, (batch_size, seq_len, dim))

        E_loss0 = 10 * torch.sqrt(MSE_loss(X, X_tilde))  

        Embedder.zero_grad()
        Recovery.zero_grad()

        E_loss0.backward(retain_graph=True)

        embedder_optimizer.step()
        recovery_optimizer.step()

        if e in range(1,epoch) and batch_index == 0:
            print('step: '+ str(e) + '/' + str(epoch) + ', e_loss: ' + str(np.sqrt(E_loss0.detach().numpy())))

  print('Finish Embedding Network Training')

  # Training only with supervised loss
  print('Start Training with Supervised Loss Only')
  for e in range(epoch): 
    for batch_index, X in enumerate(loader):

        H, _ = Embedder(X.float())
        H = torch.reshape(H, (batch_size, seq_len, hidden_dim))

        H_hat_supervise, _ = Supervisor(H)
        H_hat_supervise = torch.reshape(H_hat_supervise, (batch_size, seq_len, hidden_dim))  

        G_loss_S = MSE_loss(H[:,1:,:], H_hat_supervise[:,:-1,:])


        Embedder.zero_grad()
        Supervisor.zero_grad()

        G_loss_S.backward(retain_graph=True)

        embedder_optimizer.step()
        supervisor_optimizer.step()

        if e in range(1,epoch) and batch_index == 0:
            print('step: '+ str(e) + '/' + str(epoch) + ', s_loss: ' + str(np.sqrt(G_loss_S.detach().numpy())))

  print('Finish Training with Supervised Loss Only')
  # Joint Training
  

  print('Start Joint Training')
  for itt in range(epoch):
    for kk in range(2):
      X = next(iter(loader))
      random_data = random_generator(batch_size=batch_size, z_dim=dim, 
                                       T_mb=extract_time(data)[0], max_seq_len=extract_time(data)[1])
        
      # Generator Training 
      ## Train Generator
      z = torch.tensor(random_data)
      z = z.float()
        
      e_hat, _ = Generator(z)
      e_hat = torch.reshape(e_hat, (batch_size, seq_len, hidden_dim))
        
      H_hat, _ = Supervisor(e_hat)
      H_hat = torch.reshape(H_hat, (batch_size, seq_len, hidden_dim))
        
      Y_fake = Discriminator(H_hat)
      Y_fake = torch.reshape(Y_fake, (batch_size, seq_len, 1))
        
      x_hat, _ = Recovery(H_hat)
      x_hat = torch.reshape(x_hat, (batch_size, seq_len, dim))
        
      H, _ = Embedder(X.float())
      H = torch.reshape(H, (batch_size, seq_len, hidden_dim))

      H_hat_supervise, _ = Supervisor(H)
      H_hat_supervise = torch.reshape(H_hat_supervise, (batch_size, seq_len, hidden_dim))

      Generator.zero_grad()
      Supervisor.zero_grad()
      Discriminator.zero_grad()
      Recovery.zero_grad()

      # line 267 of original implementation: 
      # G_loss_U, G_loss_S, G_loss_V
      G_loss_S = MSE_loss(H[:,1:,:], H_hat_supervise[:,:-1,:])
      binary_cross_entropy_loss = nn.BCEWithLogitsLoss()
      # logits first, then targets
      # D_loss_real(Y_real, torch.ones_like(Y_real))
      G_loss_U = binary_cross_entropy_loss(Y_fake, torch.ones_like(Y_fake))
        
      G_loss_V1 = torch.mean(torch.abs((torch.std(x_hat, [0], unbiased = False)) + 1e-6 - (torch.std(X, [0]) + 1e-6)))
      G_loss_V2 = torch.mean(torch.abs((torch.mean(x_hat, [0]) - (torch.mean(X, [0])))))
      G_loss_V = G_loss_V1 + G_loss_V2
        
      # doing a backward step for each loss should result in gradients accumulating 
      # so we should be able to optimize them jointly
      G_loss_S.backward(retain_graph=True)#
      G_loss_U.backward(retain_graph=True)
      G_loss_V.backward(retain_graph=True)#


      generator_optimizer.step()
      supervisor_optimizer.step()
      discriminator_optimizer.step()
      # Train Embedder 
      ## line 270: we only optimize E_loss_T0
      ## E_loss_T0 = just mse of x and x_tilde
      # but it calls E_solver which optimizes E_loss, which is a sum of 
      # E_loss0 and 0.1* G_loss_S
      MSE_loss = nn.MSELoss()
        
      H, _ = Embedder(X.float())
      H = torch.reshape(H, (batch_size, seq_len, hidden_dim))

      X_tilde, _ = Recovery(H)
      X_tilde = torch.reshape(X_tilde, (batch_size, seq_len, dim))

      E_loss_T0 = MSE_loss(X, X_tilde)
      E_loss0 = 10 * torch.sqrt(MSE_loss(X, X_tilde))  
        
      H_hat_supervise, _ = Supervisor(H)
      H_hat_supervise = torch.reshape(H_hat_supervise, (batch_size, seq_len, hidden_dim))  

      G_loss_S = MSE_loss(H[:,1:,:], H_hat_supervise[:,:-1,:])
      E_loss = E_loss0  + 0.1 * G_loss_S
        
      G_loss_S.backward(retain_graph=True)
      E_loss_T0.backward()
        
      Embedder.zero_grad()
      Recovery.zero_grad()
      Supervisor.zero_grad()
        
      embedder_optimizer.step()
      recovery_optimizer.step()
      supervisor_optimizer.step()
    # train Discriminator
    for batch_index, X in enumerate(loader):
      random_data = random_generator(batch_size=batch_size, z_dim=dim, 
                                       T_mb=extract_time(data)[0], max_seq_len=extract_time(data)[1])
      
      z = torch.tensor(random_data)
      z = z.float()

      H, _ = Embedder(X)
      H = torch.reshape(H, (batch_size, seq_len, hidden_dim))

      Y_real = Discriminator(H)
      Y_real = torch.reshape(Y_real, (batch_size, seq_len, 1))
      
      e_hat, _ = Generator(z)
      e_hat = torch.reshape(e_hat, (batch_size, seq_len, hidden_dim))

      Y_fake_e = Discriminator(e_hat)
      Y_fake_e = torch.reshape(Y_fake_e, (batch_size, seq_len, 1))
        
      H_hat, _ = Supervisor(e_hat)
      H_hat = torch.reshape(H_hat, (batch_size, seq_len, hidden_dim))
        
      Y_fake = Discriminator(H_hat)
      Y_fake = torch.reshape(Y_fake, (batch_size, seq_len, 1))
        
      x_hat, _ = Recovery(H_hat)
      x_hat = torch.reshape(x_hat, (batch_size, seq_len, dim))

      Generator.zero_grad()
      Supervisor.zero_grad()
      Discriminator.zero_grad()
      Recovery.zero_grad()
      Embedder.zero_grad()

      # logits first, then targets
      # D_loss_real(Y_real, torch.ones_like(Y_real))
      D_loss_real = nn.BCEWithLogitsLoss()
      DLR = D_loss_real(Y_real, torch.ones_like(Y_real))

      D_loss_fake = nn.BCEWithLogitsLoss()
      DLF = D_loss_fake(Y_fake, torch.zeros_like(Y_fake))

      D_loss_fake_e = nn.BCEWithLogitsLoss()
      DLF_e = D_loss_fake_e(Y_fake_e, torch.zeros_like(Y_fake_e))

      D_loss = DLR + DLF + gamma * DLF_e

      # D_loss.backward(retain_graph=True)
      
      # discriminator_optimizer.step()

      # check discriminator loss before updating
      check_d_loss = D_loss
      if (check_d_loss > 0.15):
        D_loss.backward(retain_graph=True)
        discriminator_optimizer.step()        
        
      H, _ = Embedder(X.float())
      H = torch.reshape(H, (batch_size, seq_len, hidden_dim)) 
        
      X_tilde, _ = Recovery(H)
      X_tilde = torch.reshape(X_tilde, (batch_size, seq_len, dim))

      
      z = torch.tensor(random_data)
      z = z.float()
        
      e_hat, _ = Generator(z)
      e_hat = torch.reshape(e_hat, (batch_size, seq_len, hidden_dim))
        
      H_hat, _ = Supervisor(e_hat)
      H_hat = torch.reshape(H_hat, (batch_size, seq_len, hidden_dim))
        
      Y_fake = Discriminator(H_hat)
      Y_fake = torch.reshape(Y_fake, (batch_size, seq_len, 1))
        
      x_hat, _ = Recovery(H_hat)
      x_hat = torch.reshape(x_hat, (batch_size, seq_len, dim))
        
      H, _ = Embedder(X.float())
      H = torch.reshape(H, (batch_size, seq_len, hidden_dim))

      H_hat_supervise, _ = Supervisor(H)
      H_hat_supervise = torch.reshape(H_hat_supervise, (batch_size, seq_len, hidden_dim))

      G_loss_S = MSE_loss(H[:,1:,:], H_hat_supervise[:,:-1,:])
      binary_cross_entropy_loss = nn.BCEWithLogitsLoss()
      # logits first then targets
      G_loss_U = binary_cross_entropy_loss(Y_fake, torch.ones_like(Y_fake))
        
      G_loss_V1 = torch.mean(torch.abs((torch.std(x_hat, [0], unbiased = False)) + 1e-6 - (torch.std(X, [0]) + 1e-6)))
      G_loss_V2 = torch.mean(torch.abs((torch.mean(x_hat, [0]) - (torch.mean(X, [0])))))
      G_loss_V = G_loss_V1 + G_loss_V2
    
      E_loss_T0 = MSE_loss(X, X_tilde)
      E_loss0 = 10 * torch.sqrt(MSE_loss(X, X_tilde))  
      E_loss = E_loss0  + 0.1 * G_loss_S
        
      # doing a backward step for each loss should result in gradients accumulating 
      # so we should be able to optimize them jointly
      G_loss_S.backward(retain_graph=True)#
      G_loss_U.backward(retain_graph=True)
      G_loss_V.backward(retain_graph=True)#
      E_loss.backward()

      generator_optimizer.step()
      supervisor_optimizer.step()
      #discriminator_optimizer.step() 
      embedder_optimizer.step()
      recovery_optimizer.step()
            
      print('step: '+ str(itt) + '/' + str(epoch) + 
            ', D_loss: ' + str(D_loss.detach().numpy()) +
            ', G_loss_U: ' + str(G_loss_U.detach().numpy()) + 
            ', G_loss_S: ' + str(G_loss_S.detach().numpy()) + 
            ', E_loss_t0: ' + str(np.sqrt(E_loss0.detach().numpy()))
             )
  print('Finish Joint Training')

In [51]:
parameters = dict()
parameters['module'] = 'gru' 
parameters['hidden_dim'] = 40
parameters['num_layers'] = 1
parameters['iterations'] = 10000
parameters['batch_size'] = 128
parameters['epoch'] = 100

In [52]:
TimeGAN(data, parameters)

Start Embedding Network Training
step: 1/100, e_loss: 1.009767
step: 2/100, e_loss: 0.6797879
step: 3/100, e_loss: 0.4289872
step: 4/100, e_loss: 0.3894127
step: 5/100, e_loss: 0.32046777
step: 6/100, e_loss: 0.3286467
step: 7/100, e_loss: 0.3192449
step: 8/100, e_loss: 0.31118768
step: 9/100, e_loss: 0.3133793
step: 10/100, e_loss: 0.30645177
step: 11/100, e_loss: 0.34026206
step: 12/100, e_loss: 0.29621667
step: 13/100, e_loss: 0.29243845
step: 14/100, e_loss: 0.2806925
step: 15/100, e_loss: 0.295586
step: 16/100, e_loss: 0.28142008
step: 17/100, e_loss: 0.2730557
step: 18/100, e_loss: 0.292722
step: 19/100, e_loss: 0.27619803
step: 20/100, e_loss: 0.25657016
step: 21/100, e_loss: 0.26980644
step: 22/100, e_loss: 0.2649887
step: 23/100, e_loss: 0.25532895
step: 24/100, e_loss: 0.25511038
step: 25/100, e_loss: 0.26679343
step: 26/100, e_loss: 0.24637796
step: 27/100, e_loss: 0.22073944
step: 28/100, e_loss: 0.2541585
step: 29/100, e_loss: 0.22806829
step: 30/100, e_loss: 0.22751401
st

step: 0/100, D_loss: 1.8768499, G_loss_U: 1.6271858, G_loss_S: 0.018232621, E_loss_t0: 2.1577752
step: 0/100, D_loss: 1.8443937, G_loss_U: 1.6829528, G_loss_S: 0.02271131, E_loss_t0: 2.1661859
step: 0/100, D_loss: 1.8115331, G_loss_U: 1.7380332, G_loss_S: 0.028219696, E_loss_t0: 2.1405263
step: 0/100, D_loss: 1.7809184, G_loss_U: 1.7923503, G_loss_S: 0.03334316, E_loss_t0: 2.297022
step: 0/100, D_loss: 1.7454196, G_loss_U: 1.8458824, G_loss_S: 0.041067705, E_loss_t0: 2.2466483
step: 0/100, D_loss: 1.7092267, G_loss_U: 1.8986272, G_loss_S: 0.0494271, E_loss_t0: 2.2305067
step: 0/100, D_loss: 1.6716707, G_loss_U: 1.9506211, G_loss_S: 0.05831671, E_loss_t0: 2.1965141
step: 0/100, D_loss: 1.6338896, G_loss_U: 2.001928, G_loss_S: 0.06675565, E_loss_t0: 2.2452614
step: 0/100, D_loss: 1.5931315, G_loss_U: 2.0526743, G_loss_S: 0.07592331, E_loss_t0: 2.2086132
step: 0/100, D_loss: 1.5507807, G_loss_U: 2.1030416, G_loss_S: 0.08502562, E_loss_t0: 2.0756297
step: 0/100, D_loss: 1.5086648, G_loss_U

step: 1/100, D_loss: 0.14512604, G_loss_U: 4.824295, G_loss_S: 0.12985444, E_loss_t0: 2.2147906
step: 1/100, D_loss: 0.14582479, G_loss_U: 4.823802, G_loss_S: 0.12985, E_loss_t0: 2.2477758
step: 1/100, D_loss: 0.14602947, G_loss_U: 4.8233123, G_loss_S: 0.12984267, E_loss_t0: 2.1833284
step: 1/100, D_loss: 0.14617407, G_loss_U: 4.822754, G_loss_S: 0.1298197, E_loss_t0: 2.1879103
step: 1/100, D_loss: 0.14678267, G_loss_U: 4.822271, G_loss_S: 0.12969677, E_loss_t0: 2.1915693
step: 1/100, D_loss: 0.14724824, G_loss_U: 4.821692, G_loss_S: 0.1295033, E_loss_t0: 2.298774
step: 1/100, D_loss: 0.14703238, G_loss_U: 4.8212504, G_loss_S: 0.12938051, E_loss_t0: 2.1919694
step: 1/100, D_loss: 0.14776762, G_loss_U: 4.8207555, G_loss_S: 0.12912744, E_loss_t0: 2.2140126
step: 1/100, D_loss: 0.14786461, G_loss_U: 4.820287, G_loss_S: 0.12884478, E_loss_t0: 2.2601688
step: 1/100, D_loss: 0.14799559, G_loss_U: 4.819807, G_loss_S: 0.12856068, E_loss_t0: 2.184244
step: 1/100, D_loss: 0.14909583, G_loss_U: 4

step: 1/100, D_loss: 0.13661093, G_loss_U: 4.941361, G_loss_S: 0.015105347, E_loss_t0: 2.2272336
step: 1/100, D_loss: 0.13570563, G_loss_U: 4.9412293, G_loss_S: 0.015366309, E_loss_t0: 2.1796236
step: 1/100, D_loss: 0.13592309, G_loss_U: 4.9410014, G_loss_S: 0.014911703, E_loss_t0: 2.1085646
step: 1/100, D_loss: 0.13750027, G_loss_U: 4.9408937, G_loss_S: 0.014318791, E_loss_t0: 2.1798837
step: 1/100, D_loss: 0.14031023, G_loss_U: 4.9407153, G_loss_S: 0.013478633, E_loss_t0: 2.225784
step: 1/100, D_loss: 0.14494514, G_loss_U: 4.9406176, G_loss_S: 0.0124157425, E_loss_t0: 2.1520383
step: 1/100, D_loss: 0.15149969, G_loss_U: 4.9646506, G_loss_S: 0.01138593, E_loss_t0: 2.2776413
step: 1/100, D_loss: 0.1390528, G_loss_U: 4.9645066, G_loss_S: 0.0116201425, E_loss_t0: 2.1474771
step: 1/100, D_loss: 0.13820717, G_loss_U: 4.9643044, G_loss_S: 0.011680777, E_loss_t0: 2.1462731
step: 1/100, D_loss: 0.13820116, G_loss_U: 4.964096, G_loss_S: 0.011611914, E_loss_t0: 2.2225742
step: 1/100, D_loss: 0.

step: 2/100, D_loss: 0.147481, G_loss_U: 4.605766, G_loss_S: 0.0041217636, E_loss_t0: 2.1710782
step: 2/100, D_loss: 0.14756091, G_loss_U: 4.605928, G_loss_S: 0.0041806847, E_loss_t0: 2.2634149
step: 2/100, D_loss: 0.1476443, G_loss_U: 4.6056185, G_loss_S: 0.004129402, E_loss_t0: 2.2826927
step: 2/100, D_loss: 0.14768904, G_loss_U: 4.605585, G_loss_S: 0.0040986156, E_loss_t0: 2.1543634
step: 2/100, D_loss: 0.14788297, G_loss_U: 4.6052756, G_loss_S: 0.0040173917, E_loss_t0: 2.226509
step: 2/100, D_loss: 0.14823836, G_loss_U: 4.605252, G_loss_S: 0.0040237317, E_loss_t0: 2.2380874
step: 2/100, D_loss: 0.14833577, G_loss_U: 4.6050854, G_loss_S: 0.0039966414, E_loss_t0: 2.281452
step: 2/100, D_loss: 0.1487397, G_loss_U: 4.6050224, G_loss_S: 0.0039591477, E_loss_t0: 2.2284164
step: 2/100, D_loss: 0.1489447, G_loss_U: 4.6048675, G_loss_S: 0.003911008, E_loss_t0: 2.1384816
step: 2/100, D_loss: 0.14931194, G_loss_U: 4.604903, G_loss_S: 0.003865579, E_loss_t0: 2.2210238
step: 2/100, D_loss: 0.14

...

step: 96/100, D_loss: 0.16688327, G_loss_U: 3.0527744, G_loss_S: 9.054297e-06, E_loss_t0: 2.1832554
step: 96/100, D_loss: 0.16848238, G_loss_U: 3.048527, G_loss_S: 6.474225e-06, E_loss_t0: 2.2557902
step: 96/100, D_loss: 0.16910978, G_loss_U: 3.0501034, G_loss_S: 1.0849484e-05, E_loss_t0: 2.2135983
step: 96/100, D_loss: 0.16885424, G_loss_U: 3.0569162, G_loss_S: 1.7116752e-05, E_loss_t0: 2.176861
step: 96/100, D_loss: 0.16782144, G_loss_U: 3.0684087, G_loss_S: 2.172676e-05, E_loss_t0: 2.1332285
step: 96/100, D_loss: 0.16609943, G_loss_U: 3.0840561, G_loss_S: 2.1779118e-05, E_loss_t0: 2.1578531
step: 96/100, D_loss: 0.1637786, G_loss_U: 3.1033666, G_loss_S: 2.7069029e-05, E_loss_t0: 2.1680207
step: 96/100, D_loss: 0.16095997, G_loss_U: 3.125882, G_loss_S: 3.123875e-05, E_loss_t0: 2.25326
step: 96/100, D_loss: 0.15773572, G_loss_U: 3.1511757, G_loss_S: 3.5356792e-05, E_loss_t0: 2.1802983
step: 96/100, D_loss: 0.15418579, G_loss_U: 3.178857, G_loss_S: 3.9755665e-05, E_loss_t0: 2.2473671
s

step: 96/100, D_loss: 0.14649238, G_loss_U: 3.208566, G_loss_S: 4.787913e-07, E_loss_t0: 2.1037495
step: 96/100, D_loss: 0.14649318, G_loss_U: 3.208566, G_loss_S: 5.489733e-07, E_loss_t0: 2.1912675
step: 96/100, D_loss: 0.14649394, G_loss_U: 3.208566, G_loss_S: 5.306787e-07, E_loss_t0: 2.2144518
step: 96/100, D_loss: 0.14649312, G_loss_U: 3.208566, G_loss_S: 5.116667e-07, E_loss_t0: 2.1308155
step: 96/100, D_loss: 0.14649276, G_loss_U: 3.208566, G_loss_S: 4.684215e-07, E_loss_t0: 2.1501415
step: 96/100, D_loss: 0.14649436, G_loss_U: 3.208566, G_loss_S: 4.7400493e-07, E_loss_t0: 2.1140258
step: 96/100, D_loss: 0.14649619, G_loss_U: 3.208566, G_loss_S: 5.1608635e-07, E_loss_t0: 2.29705
step: 96/100, D_loss: 0.14649469, G_loss_U: 3.208566, G_loss_S: 4.911595e-07, E_loss_t0: 2.2308931
step: 96/100, D_loss: 0.14649358, G_loss_U: 3.208566, G_loss_S: 4.843569e-07, E_loss_t0: 2.2397983
step: 96/100, D_loss: 0.14649396, G_loss_U: 3.208566, G_loss_S: 4.8938887e-07, E_loss_t0: 2.1373825
step: 96/

step: 97/100, D_loss: 0.14886202, G_loss_U: 3.1901653, G_loss_S: 5.8442896e-07, E_loss_t0: 2.2190406
step: 97/100, D_loss: 0.14886224, G_loss_U: 3.1901653, G_loss_S: 5.9429817e-07, E_loss_t0: 2.2854576
step: 97/100, D_loss: 0.14885965, G_loss_U: 3.1901653, G_loss_S: 5.388057e-07, E_loss_t0: 2.1989853
step: 97/100, D_loss: 0.14886056, G_loss_U: 3.1901653, G_loss_S: 6.035424e-07, E_loss_t0: 2.1739793
step: 97/100, D_loss: 0.14886026, G_loss_U: 3.1901653, G_loss_S: 5.0629495e-07, E_loss_t0: 2.230946
step: 97/100, D_loss: 0.14886168, G_loss_U: 3.1901653, G_loss_S: 5.9846974e-07, E_loss_t0: 2.188401
step: 97/100, D_loss: 0.14886111, G_loss_U: 3.1901653, G_loss_S: 5.323331e-07, E_loss_t0: 2.2444785
step: 97/100, D_loss: 0.1488584, G_loss_U: 3.1901653, G_loss_S: 5.457002e-07, E_loss_t0: 2.072516
step: 97/100, D_loss: 0.14886093, G_loss_U: 3.1901653, G_loss_S: 5.4277086e-07, E_loss_t0: 2.1630397
step: 97/100, D_loss: 0.14886211, G_loss_U: 3.1901648, G_loss_S: 4.72298e-07, E_loss_t0: 2.2270787


step: 98/100, D_loss: 0.14865878, G_loss_U: 3.1910534, G_loss_S: 9.4286935e-07, E_loss_t0: 2.2796648
step: 98/100, D_loss: 0.14866175, G_loss_U: 3.1910534, G_loss_S: 8.1748215e-07, E_loss_t0: 2.1807451
step: 98/100, D_loss: 0.14866182, G_loss_U: 3.1910534, G_loss_S: 7.752567e-07, E_loss_t0: 2.247123
step: 98/100, D_loss: 0.14866361, G_loss_U: 3.1910534, G_loss_S: 7.3256393e-07, E_loss_t0: 2.2289128
step: 98/100, D_loss: 0.14866754, G_loss_U: 3.1910534, G_loss_S: 6.983354e-07, E_loss_t0: 2.2206538
step: 98/100, D_loss: 0.14866644, G_loss_U: 3.1910534, G_loss_S: 6.6925423e-07, E_loss_t0: 2.1905944
step: 98/100, D_loss: 0.1486658, G_loss_U: 3.1910534, G_loss_S: 6.260163e-07, E_loss_t0: 2.2234757
step: 98/100, D_loss: 0.1486683, G_loss_U: 3.1910534, G_loss_S: 6.776288e-07, E_loss_t0: 2.2106884
step: 98/100, D_loss: 0.14866924, G_loss_U: 3.1910534, G_loss_S: 6.194264e-07, E_loss_t0: 2.2147405
step: 98/100, D_loss: 0.14867146, G_loss_U: 3.1910534, G_loss_S: 6.578401e-07, E_loss_t0: 2.2228346

step: 99/100, D_loss: 0.14654021, G_loss_U: 3.2066278, G_loss_S: 4.6266796e-06, E_loss_t0: 2.1729102
step: 99/100, D_loss: 0.14653549, G_loss_U: 3.2066278, G_loss_S: 3.948704e-06, E_loss_t0: 2.2125573
step: 99/100, D_loss: 0.14653346, G_loss_U: 3.2066278, G_loss_S: 2.7081358e-06, E_loss_t0: 2.2379017
step: 99/100, D_loss: 0.14653271, G_loss_U: 3.2066278, G_loss_S: 2.4961078e-06, E_loss_t0: 2.125583
step: 99/100, D_loss: 0.14652596, G_loss_U: 3.2066278, G_loss_S: 1.5861914e-06, E_loss_t0: 2.1748924
step: 99/100, D_loss: 0.14651822, G_loss_U: 3.2066278, G_loss_S: 1.6650258e-06, E_loss_t0: 2.1909106
step: 99/100, D_loss: 0.14651744, G_loss_U: 3.2066278, G_loss_S: 1.2358514e-06, E_loss_t0: 2.132839
step: 99/100, D_loss: 0.1465179, G_loss_U: 3.2066278, G_loss_S: 1.4626502e-06, E_loss_t0: 2.1786387
step: 99/100, D_loss: 0.14651535, G_loss_U: 3.2066278, G_loss_S: 1.3455458e-06, E_loss_t0: 2.124536
step: 99/100, D_loss: 0.1465102, G_loss_U: 3.2066278, G_loss_S: 1.4925388e-06, E_loss_t0: 2.0158

In [ ]:
from utils import extract_time

In [56]:
random_test = random_generator(12800, dim, extract_time(data)[0], extract_time(data)[1])

In [36]:
torch.tensor(random_test).shape

torch.Size([12800, 24, 1])

Oben wird die selbe anzahl an Noise-Daten wie in den echten Daten erzeugt, und dann in den Generator geworfen; Das wird dann wiederrum in die Recovery gegeben, und darauf dann die Visualisierung gemacht

In [57]:
test = Generator(torch.tensor(random_generator(12800, dim, extract_time(data)[0], extract_time(data)[1])).float())[0]

In [58]:
test.shape

torch.Size([307200, 20])

In [59]:
test = torch.reshape(test, (12800, seq_len, 20))

In [45]:
test.shape

torch.Size([12800, 24, 20])

In [60]:
test_recovery = Recovery(test)

In [61]:
test_recovery = torch.reshape(test_recovery[0], (12800, seq_len, dim))

In [62]:
test_recovery.shape

torch.Size([12800, 24, 1])

In [42]:
"""Time-series Generative Adversarial Networks (TimeGAN) Codebase.
Reference: Jinsung Yoon, Daniel Jarrett, Mihaela van der Schaar, 
"Time-series Generative Adversarial Networks," 
Neural Information Processing Systems (NeurIPS), 2019.
Paper link: https://papers.nips.cc/paper/8789-time-series-generative-adversarial-networks
Last updated Date: April 24th 2020
Code author: Jinsung Yoon (jsyoon0823@gmail.com)
-----------------------------
visualization_metrics.py
Note: Use PCA or tSNE for generated and original data visualization
"""

# Necessary packages
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np

   
def visualization (ori_data, generated_data, analysis):
  """Using PCA or tSNE for generated and original data visualization.
  
  Args:
    - ori_data: original data
    - generated_data: generated synthetic data
    - analysis: tsne or pca
  """  
  # Analysis sample size (for faster computation)
  anal_sample_no = min([1000, len(ori_data)])
  idx = np.random.permutation(len(ori_data))[:anal_sample_no]
    
  # Data preprocessing
  ori_data = np.asarray(ori_data)
  generated_data = np.asarray(generated_data)  
  
  ori_data = ori_data[idx]
  generated_data = generated_data[idx]
  
  no, seq_len, dim = ori_data.shape  
  
  for i in range(anal_sample_no):
    if (i == 0):
      prep_data = np.reshape(np.mean(ori_data[0,:,:], 1), [1,seq_len])
      prep_data_hat = np.reshape(np.mean(generated_data[0,:,:],1), [1,seq_len])
    else:
      prep_data = np.concatenate((prep_data, 
                                  np.reshape(np.mean(ori_data[i,:,:],1), [1,seq_len])))
      prep_data_hat = np.concatenate((prep_data_hat, 
                                      np.reshape(np.mean(generated_data[i,:,:],1), [1,seq_len])))
    
  # Visualization parameter        
  colors = ["red" for i in range(anal_sample_no)] + ["blue" for i in range(anal_sample_no)]    
    
  if analysis == 'pca':
    # PCA Analysis
    pca = PCA(n_components = 2)
    pca.fit(prep_data)
    pca_results = pca.transform(prep_data)
    pca_hat_results = pca.transform(prep_data_hat)
    
    # Plotting
    f, ax = plt.subplots(1)    
    plt.scatter(pca_results[:,0], pca_results[:,1],
                c = colors[:anal_sample_no], alpha = 0.2, label = "Original")
    plt.scatter(pca_hat_results[:,0], pca_hat_results[:,1], 
                c = colors[anal_sample_no:], alpha = 0.2, label = "Synthetic")
  
    ax.legend()  
    plt.title('PCA plot')
    plt.xlabel('x-pca')
    plt.ylabel('y_pca')
    plt.show()
    
  elif analysis == 'tsne':
    
    # Do t-SNE Analysis together       
    prep_data_final = np.concatenate((prep_data, prep_data_hat), axis = 0)
    
    # TSNE anlaysis
    tsne = TSNE(n_components = 2, verbose = 1, perplexity = 40, n_iter = 300)
    tsne_results = tsne.fit_transform(prep_data_final)
      
    # Plotting
    f, ax = plt.subplots(1)
      
    plt.scatter(tsne_results[:anal_sample_no,0], tsne_results[:anal_sample_no,1], 
                c = colors[:anal_sample_no], alpha = 0.2, label = "Original")
    plt.scatter(tsne_results[anal_sample_no:,0], tsne_results[anal_sample_no:,1], 
                c = colors[anal_sample_no:], alpha = 0.2, label = "Synthetic")
  
    ax.legend()
      
    plt.title('t-SNE plot')
    plt.xlabel('x-tsne')
    plt.ylabel('y_tsne')
    plt.show()    

In [63]:
visualization(data, test_recovery.detach().numpy(), 'tsne')

[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 2000 samples in 0.000s...
[t-SNE] Computed neighbors for 2000 samples in 0.075s...
[t-SNE] Computed conditional probabilities for sample 1000 / 2000
[t-SNE] Computed conditional probabilities for sample 2000 / 2000
[t-SNE] Mean sigma: 0.000021
[t-SNE] KL divergence after 250 iterations with early exaggeration: 60.854031
[t-SNE] KL divergence after 300 iterations: 0.926143


In [64]:
visualization(data, test_recovery.detach().numpy(), 'pca')

In [81]:
fig, ax = plt.subplots()
ax1 = plt.plot(test[0].detach().numpy())


In [83]:
fig, ax = plt.subplots()
ax1 = plt.plot(test[5000].detach().numpy())

In [82]:
test[0].shape

torch.Size([24, 20])

In [80]:
test[5000]

tensor([[0.5021, 0.4522, 0.5137, 0.5533, 0.5172, 0.4806, 0.5186, 0.5120, 0.4906,
         0.5256, 0.5015, 0.5336, 0.4800, 0.4720, 0.4694, 0.5361, 0.4928, 0.4256,
         0.5069, 0.5053],
        [0.5072, 0.4507, 0.5135, 0.5548, 0.5160, 0.4792, 0.5276, 0.5133, 0.4835,
         0.5326, 0.4997, 0.5383, 0.4747, 0.4693, 0.4642, 0.5378, 0.4944, 0.4157,
         0.5215, 0.4998],
        [0.5099, 0.4481, 0.5158, 0.5547, 0.5158, 0.4784, 0.5348, 0.5135, 0.4787,
         0.5365, 0.4994, 0.5436, 0.4706, 0.4677, 0.4634, 0.5384, 0.4963, 0.4122,
         0.5309, 0.4979],
        [0.5120, 0.4459, 0.5183, 0.5545, 0.5161, 0.4781, 0.5394, 0.5137, 0.4763,
         0.5379, 0.4994, 0.5477, 0.4674, 0.4667, 0.4640, 0.5390, 0.4979, 0.4114,
         0.5364, 0.4971],
        [0.5131, 0.4439, 0.5208, 0.5543, 0.5166, 0.4786, 0.5423, 0.5138, 0.4751,
         0.5381, 0.4994, 0.5506, 0.4650, 0.4666, 0.4652, 0.5390, 0.4993, 0.4119,
         0.5395, 0.4968],
        [0.5131, 0.4419, 0.5232, 0.5539, 0.5170, 0.4796, 0.5

In [75]:
fig, ax = plt.subplots()
ax1 = plt.plot(data[0].detach().numpy())


In [66]:
test_recovery[0]

tensor([[0.5207],
        [0.5224],
        [0.5233],
        [0.5236],
        [0.5238],
        [0.5239],
        [0.5240],
        [0.5240],
        [0.5240],
        [0.5240],
        [0.5241],
        [0.5241],
        [0.5241],
        [0.5241],
        [0.5241],
        [0.5241],
        [0.5241],
        [0.5241],
        [0.5241],
        [0.5241],
        [0.5241],
        [0.5242],
        [0.5242],
        [0.5242]], grad_fn=<SelectBackward>)

In [78]:
test_recovery[5000]

tensor([[0.5207],
        [0.5224],
        [0.5233],
        [0.5236],
        [0.5238],
        [0.5239],
        [0.5239],
        [0.5240],
        [0.5240],
        [0.5240],
        [0.5241],
        [0.5241],
        [0.5241],
        [0.5241],
        [0.5241],
        [0.5241],
        [0.5241],
        [0.5241],
        [0.5241],
        [0.5241],
        [0.5241],
        [0.5241],
        [0.5241],
        [0.5241]], grad_fn=<SelectBackward>)

In [74]:
test_recovery

tensor([[[0.5207],
         [0.5224],
         [0.5233],
         ...,
         [0.5242],
         [0.5242],
         [0.5242]],

        [[0.5207],
         [0.5224],
         [0.5232],
         ...,
         [0.5241],
         [0.5241],
         [0.5242]],

        [[0.5207],
         [0.5224],
         [0.5232],
         ...,
         [0.5242],
         [0.5242],
         [0.5242]],

        ...,

        [[0.5207],
         [0.5224],
         [0.5233],
         ...,
         [0.5241],
         [0.5241],
         [0.5241]],

        [[0.5207],
         [0.5224],
         [0.5233],
         ...,
         [0.5241],
         [0.5242],
         [0.5242]],

        [[0.5207],
         [0.5224],
         [0.5232],
         ...,
         [0.5241],
         [0.5241],
         [0.5241]]], grad_fn=<ViewBackward>)